In [3]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [5]:
subject = 'Mato Grosso - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [10]:
data = pd.read_csv('../Inputs Atuais/2008_01_model_input_MT.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data = data.drop('Mato Grosso - Produção de Cimento (t)', axis=1)
data

,Ano,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado,Mato Grosso - IDH,Mato Grosso - IDH Longevidade,Mato Grosso - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Mato Grosso - IDH Renda,Mato Grosso - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso - Consumo de Cimento (t)
0,2008,6.535602e+07,2.793553e+06,17.127903,5.260918e+07,0.752766,0.787367,0.732454,5.483335e+08,2.546644e+09,...,0.738916,8.063005,31.686523,993.754162,1421.360784,26.585520,19.247754,68.306560,1.265376,994.957000
1,2009,6.593967e+07,2.815133e+06,17.131657,5.267158e+07,0.753704,0.788247,0.735581,5.302764e+08,2.679945e+09,...,0.738104,8.095019,26.992644,992.577520,971.624734,26.514656,18.393406,65.147890,1.139212,998.465000
2,2010,6.789156e+07,2.861375e+06,17.294037,5.339572e+07,0.754460,0.788564,0.738478,5.046425e+08,2.891930e+09,...,0.737478,8.057848,25.116066,989.658472,896.219335,26.516966,17.684947,62.460338,1.378783,1099.340000
3,2011,6.985811e+07,2.887266e+06,17.471861,5.415240e+07,0.755039,0.788328,0.741142,4.854622e+08,3.091988e+09,...,0.737032,8.047069,28.216671,987.048009,916.320612,26.594241,18.048644,63.561988,1.548197,1204.074000
4,2012,7.159016e+07,2.905726e+06,17.591515,5.472408e+07,0.755027,0.787468,0.742721,4.867661e+08,3.189506e+09,...,0.736572,6.695389,27.946087,986.033104,884.118586,26.504045,18.419653,65.231377,1.625213,1381.539000
5,2013,7.314074e+07,2.912179e+06,17.684470,5.519726e+07,0.756113,0.786174,0.746782,5.225848e+08,3.093872e+09,...,0.736953,7.068881,31.557800,985.745432,817.482501,26.560721,18.794569,66.303790,1.668395,1618.657000
6,2014,7.450695e+07,2.898181e+06,17.765377,5.559725e+07,0.755956,0.784438,0.748013,5.676413e+08,2.779340e+09,...,0.736866,7.495401,33.403862,985.582958,728.555092,26.792583,18.998037,66.249639,1.689375,1411.234429
7,2015,7.552129e+07,2.867943e+06,17.804826,5.583983e+07,0.755452,0.782141,0.749014,5.179392e+08,2.561731e+09,...,0.736688,7.838256,31.029765,985.297803,631.112819,26.741398,18.889658,66.109773,1.670487,1241.100343
8,2016,7.631332e+07,2.837856e+06,17.844276,5.605555e+07,0.758777,0.788475,0.756866,4.524144e+08,2.516859e+09,...,0.732916,7.730867,27.875013,985.139748,546.211949,26.647262,18.703122,65.821838,1.617712,1083.128229
9,2017,7.659073e+07,2.826355e+06,17.841399,5.610212e+07,0.757848,0.785609,0.756036,4.213689e+08,2.497710e+09,...,0.733636,7.251143,26.025417,985.051490,572.018116,26.728560,18.583504,65.279143,1.559939,1220.058000


In [11]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado,Mato Grosso - IDH,Mato Grosso - IDH Longevidade,Mato Grosso - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Mato Grosso - IDH Renda,Mato Grosso - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,6.535602e+07,2.793553e+06,17.127903,5.260918e+07,0.752766,0.787367,0.732454,5.483335e+08,2.546644e+09,1.874883e+10,...,11.760983,0.738916,8.063005,31.686523,993.754162,1421.360784,26.585520,19.247754,68.306560,1.265376
1,6.593967e+07,2.815133e+06,17.131657,5.267158e+07,0.753704,0.788247,0.735581,5.302764e+08,2.679945e+09,2.055015e+10,...,9.472585,0.738104,8.095019,26.992644,992.577520,971.624734,26.514656,18.393406,65.147890,1.139212
2,6.789156e+07,2.861375e+06,17.294037,5.339572e+07,0.754460,0.788564,0.738478,5.046425e+08,2.891930e+09,2.342247e+10,...,9.332485,0.737478,8.057848,25.116066,989.658472,896.219335,26.516966,17.684947,62.460338,1.378783
3,6.985811e+07,2.887266e+06,17.471861,5.415240e+07,0.755039,0.788328,0.741142,4.854622e+08,3.091988e+09,2.604265e+10,...,10.985689,0.737032,8.047069,28.216671,987.048009,916.320612,26.594241,18.048644,63.561988,1.548197
4,7.159016e+07,2.905726e+06,17.591515,5.472408e+07,0.755027,0.787468,0.742721,4.867661e+08,3.189506e+09,2.708753e+10,...,8.159014,0.736572,6.695389,27.946087,986.033104,884.118586,26.504045,18.419653,65.231377,1.625213
5,7.314074e+07,2.912179e+06,17.684470,5.519726e+07,0.756113,0.786174,0.746782,5.225848e+08,3.093872e+09,2.523394e+10,...,7.913561,0.736953,7.068881,31.557800,985.745432,817.482501,26.560721,18.794569,66.303790,1.668395
6,7.450695e+07,2.898181e+06,17.765377,5.559725e+07,0.755956,0.784438,0.748013,5.676413e+08,2.779340e+09,1.954361e+10,...,10.350442,0.736866,7.495401,33.403862,985.582958,728.555092,26.792583,18.998037,66.249639,1.689375
7,7.552129e+07,2.867943e+06,17.804826,5.583983e+07,0.755452,0.782141,0.749014,5.179392e+08,2.561731e+09,1.327596e+10,...,12.454398,0.736688,7.838256,31.029765,985.297803,631.112819,26.741398,18.889658,66.109773,1.670487
8,7.631332e+07,2.837856e+06,17.844276,5.605555e+07,0.758777,0.788475,0.756866,4.524144e+08,2.516859e+09,1.011370e+10,...,13.111908,0.732916,7.730867,27.875013,985.139748,546.211949,26.647262,18.703122,65.821838,1.617712
9,7.659073e+07,2.826355e+06,17.841399,5.610212e+07,0.757848,0.785609,0.756036,4.213689e+08,2.497710e+09,8.456309e+09,...,9.479812,0.733636,7.251143,26.025417,985.051490,572.018116,26.728560,18.583504,65.279143,1.559939


In [12]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      998.465000
1     1099.340000
2     1204.074000
3     1381.539000
4     1618.657000
5     1411.234429
6     1241.100343
7     1083.128229
8     1220.058000
9     1316.386000
10    1446.064000
11    1555.266000
12    1788.146000
13    1947.017000
14            NaN
Name: Mato Grosso - Consumo de Cimento (t), dtype: float64

In [13]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado,Mato Grosso - IDH,Mato Grosso - IDH Longevidade,Mato Grosso - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Mato Grosso - IDH Renda,Mato Grosso - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-1.740588,-1.627429,-1.768928,-1.746450,-1.736363,0.561629,-1.727009,1.204752,-0.990234,0.166091,...,0.984445,1.450893,1.102745,1.214956,2.196775,2.448959,-0.498590,1.635424,1.610412,-1.533856
1,-1.598275,-1.055001,-1.755010,-1.698578,-1.152572,0.837117,-1.343029,0.820657,-0.546688,0.428911,...,-0.051710,1.051022,1.170358,-0.661837,1.783361,0.503680,-0.709406,-0.183740,0.176596,-2.299018
2,-1.122342,0.171630,-1.152996,-1.143071,-0.682504,0.936364,-0.987463,0.275392,0.158672,0.847995,...,-0.115145,0.743048,1.091854,-1.412165,0.757751,0.177523,-0.702533,-1.692261,-1.043365,-0.846063
3,-0.642835,0.858410,-0.493731,-0.562596,-0.322949,0.862562,-0.660386,-0.132596,0.824346,1.230291,...,0.633403,0.523822,1.069089,-0.172424,-0.159437,0.264469,-0.472647,-0.917841,-0.543293,0.181401
4,-0.220508,1.348072,-0.050124,-0.124039,-0.329912,0.593304,-0.466463,-0.104860,1.148829,1.382743,...,-0.646476,0.297449,-1.785600,-0.280614,-0.516025,0.125183,-0.740974,-0.127851,0.214493,0.648489
5,0.157572,1.519248,0.294500,0.238952,0.345134,0.188062,0.031981,0.657046,0.830615,1.112297,...,-0.757614,0.484935,-0.996801,1.163487,-0.617099,-0.163043,-0.572364,0.670458,0.701294,0.910382
6,0.490699,1.147933,0.594458,0.545796,0.247355,-0.355312,0.183203,1.615453,-0.215960,0.282052,...,0.345772,0.442014,-0.096011,1.901614,-0.674184,-0.547688,0.117409,1.103701,0.676713,1.037619
7,0.738027,0.345838,0.740713,0.731890,-0.065842,-1.074422,0.306105,0.558230,-0.940033,-0.632425,...,1.298413,0.354512,0.628086,0.952359,-0.774374,-0.969163,-0.034864,0.872930,0.613224,0.923072
8,0.931149,-0.452241,0.886969,0.897373,2.001700,0.908478,1.270060,-0.835563,-1.089341,-1.093812,...,1.596124,-1.501484,0.401284,-0.309032,-0.829906,-1.336391,-0.314910,0.475739,0.482521,0.603000
9,0.998790,-0.757316,0.876303,0.933102,1.423819,0.011185,1.168125,-1.495939,-1.153056,-1.335633,...,-0.048438,-1.147363,-0.611873,-1.048571,-0.860916,-1.224770,-0.073054,0.221037,0.236176,0.252619


In [14]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0      998.465000
1     1099.340000
2     1204.074000
3     1381.539000
4     1618.657000
5     1411.234429
6     1241.100343
7     1083.128229
8     1220.058000
9     1316.386000
10    1446.064000
11    1555.266000
Name: Mato Grosso - Consumo de Cimento (t), dtype: float64

In [15]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado,Mato Grosso - IDH,Mato Grosso - IDH Longevidade,Mato Grosso - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Mato Grosso - IDH Renda,Mato Grosso - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,0.765494,-0.551552,0.800035,0.794933,-1.569273,-2.551374,0.831359,1.108405,2.635823,-0.189959,...,-2.260447,-1.55722,-1.265069,-0.730185,0.611707,1.620843,2.316005,-1.810579,-2.274914,0.168633


In [16]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    1788.146
Name: Mato Grosso - Consumo de Cimento (t), dtype: float64

In [27]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-2:], train_target.iloc[-2:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [28]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [29]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[1774549079, 1269082515, 2839315008, 683210841, 1708624696, 3403786125, 1037055006, 666940540, 1141481553, 2600681079, 1360073576, 2028522970, 2211313747, 2157469156, 1964837630, 3449293588, 278847558, 2284067725, 4023784159, 2566760638, 574785837, 959866864, 1550224023, 1440734688, 1517741161, 269073846, 2531634409, 58186430, 4262764945, 315688967, 2545910952, 2016076036, 3820369157, 3143288345, 433069138, 2351098589, 1646683030, 3550422284, 1026097175, 3114672788, 3167423497, 667530421, 1269950099, 1235838345, 2147185670, 1031142172, 1002714352, 1722745130, 1864499599, 2135709357, 1889723735, 1044018564, 2588965924, 701287185, 2548732802, 836354028, 2872875021, 2683839771, 2937378774, 2230461390, 1560128144, 2860000143, 1301651125, 2264862940, 3233247740, 2338779421, 1293831712, 3697974289, 4143624281, 3004451302, 1971599501, 1593336600, 2332376500, 2971989614, 2810561402, 1659199099, 1818527299, 1320947687, 3843581158, 3897017800, 3630196914, 3768570601, 700825571, 1392013884, 10106

val_loss: 5733.03271484375


Step: 87 ___________________________________________
val_loss: 7284.6513671875


Step: 88 ___________________________________________
val_loss: 10393.5087890625


Step: 89 ___________________________________________
val_loss: 38754.73046875


Step: 90 ___________________________________________
val_loss: 20748.46484375


Step: 91 ___________________________________________
val_loss: 32496.46484375


Step: 92 ___________________________________________
val_loss: 7416.05517578125


Step: 93 ___________________________________________
val_loss: 16404.73828125


Step: 94 ___________________________________________
val_loss: 31674.30859375


Step: 95 ___________________________________________
val_loss: 11028.56640625


Step: 96 ___________________________________________
val_loss: 24838.703125


Step: 97 ___________________________________________
val_loss: 5015.2177734375


Step: 98 ___________________________________________
val_loss: 30753.765625


Step: 99 

In [30]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 462ms/step - loss: 1718538.0000 - val_loss: 2183372.7500
Epoch 2/10000
1/1 [==============================] - 0s 15ms/step - loss: 1703498.5000 - val_loss: 2374993.0000
Epoch 3/10000
1/1 [==============================] - 0s 14ms/step - loss: 1660557.0000 - val_loss: 2643962.5000
Epoch 4/10000
1/1 [==============================] - 0s 13ms/step - loss: 1606413.0000 - val_loss: 2492512.5000
Epoch 5/10000
1/1 [==============================] - 0s 14ms/step - loss: 1577204.0000 - val_loss: 2112699.5000
Epoch 6/10000
1/1 [==============================] - 0s 14ms/step - loss: 1581459.5000 - val_loss: 2229367.5000
Epoch 7/10000
1/1 [==============================] - 0s 14ms/step - loss: 1549186.3750 - val_loss: 1910599.5000
Epoch 8/10000
1/1 [==============================] - 0s 14ms/step - loss: 1488352.0000 - val_loss: 1688057.2500
Epoch 9/10000
1/1 [==============================] - 0s 14ms/step - loss: 1464218.6250 - val_loss: 1584

1/1 [==============================] - 0s 13ms/step - loss: 399810.5312 - val_loss: 169885.5312
Epoch 75/10000
1/1 [==============================] - 0s 13ms/step - loss: 386091.5000 - val_loss: 207099.9688
Epoch 76/10000
1/1 [==============================] - 0s 14ms/step - loss: 292181.6250 - val_loss: 224599.0625
Epoch 77/10000
1/1 [==============================] - 0s 14ms/step - loss: 337209.0938 - val_loss: 254399.2812
Epoch 78/10000
1/1 [==============================] - 0s 13ms/step - loss: 295167.1250 - val_loss: 226077.3438
Epoch 79/10000
1/1 [==============================] - 0s 13ms/step - loss: 366884.3750 - val_loss: 290311.8438
Epoch 80/10000
1/1 [==============================] - 0s 14ms/step - loss: 259185.8281 - val_loss: 276847.3438
Epoch 81/10000
1/1 [==============================] - 0s 14ms/step - loss: 318529.1250 - val_loss: 278560.3750
Epoch 82/10000
1/1 [==============================] - 0s 14ms/step - loss: 234401.2031 - val_loss: 181129.4688
Epoch 83/10000
1

1/1 [==============================] - 0s 14ms/step - loss: 131305.6094 - val_loss: 59894.7656
Epoch 149/10000
1/1 [==============================] - 0s 14ms/step - loss: 143551.6094 - val_loss: 24815.9316
Epoch 150/10000
1/1 [==============================] - 0s 13ms/step - loss: 151487.7500 - val_loss: 27074.4922
Epoch 151/10000
1/1 [==============================] - 0s 13ms/step - loss: 130473.2266 - val_loss: 14310.1475
Epoch 152/10000
1/1 [==============================] - 0s 13ms/step - loss: 192858.5000 - val_loss: 15779.3506
Epoch 153/10000
1/1 [==============================] - 0s 13ms/step - loss: 145340.7969 - val_loss: 30795.8926
Epoch 154/10000
1/1 [==============================] - 0s 13ms/step - loss: 89244.0312 - val_loss: 37913.7812
Epoch 155/10000
1/1 [==============================] - 0s 13ms/step - loss: 98245.5859 - val_loss: 28923.9531
Epoch 156/10000
1/1 [==============================] - 0s 14ms/step - loss: 115169.8672 - val_loss: 39491.4453
Epoch 157/10000
1/1

1/1 [==============================] - 0s 13ms/step - loss: 46119.0000 - val_loss: 13099.3789
Epoch 223/10000
1/1 [==============================] - 0s 13ms/step - loss: 47506.7383 - val_loss: 11143.0547
Epoch 224/10000
1/1 [==============================] - 0s 14ms/step - loss: 45502.4023 - val_loss: 12361.2002
Epoch 225/10000
1/1 [==============================] - 0s 13ms/step - loss: 65901.6562 - val_loss: 11612.9658
Epoch 226/10000
1/1 [==============================] - 0s 13ms/step - loss: 39378.6367 - val_loss: 9357.3574
Epoch 227/10000
1/1 [==============================] - 0s 13ms/step - loss: 48178.3008 - val_loss: 7945.7993
Epoch 228/10000
1/1 [==============================] - 0s 13ms/step - loss: 42372.6016 - val_loss: 7883.0737
Epoch 229/10000
1/1 [==============================] - 0s 13ms/step - loss: 43398.2344 - val_loss: 6215.5850
Epoch 230/10000
1/1 [==============================] - 0s 14ms/step - loss: 52642.8867 - val_loss: 7986.2368
Epoch 231/10000
1/1 [==========

1/1 [==============================] - 0s 14ms/step - loss: 36524.9609 - val_loss: 3351.9954
Epoch 298/10000
1/1 [==============================] - 0s 14ms/step - loss: 28623.2031 - val_loss: 4060.4438
Epoch 299/10000
1/1 [==============================] - 0s 13ms/step - loss: 40620.9570 - val_loss: 4488.6904
Epoch 300/10000
1/1 [==============================] - 0s 13ms/step - loss: 41610.4609 - val_loss: 3113.1350
Epoch 301/10000
1/1 [==============================] - 0s 13ms/step - loss: 31247.5293 - val_loss: 3113.5500
Epoch 302/10000
1/1 [==============================] - 0s 13ms/step - loss: 38234.8711 - val_loss: 3048.5549
Epoch 303/10000
1/1 [==============================] - 0s 13ms/step - loss: 33682.9961 - val_loss: 3624.9167
Epoch 304/10000
1/1 [==============================] - 0s 14ms/step - loss: 36255.5586 - val_loss: 4080.4253
Epoch 305/10000
1/1 [==============================] - 0s 13ms/step - loss: 39012.8359 - val_loss: 8418.4883
Epoch 306/10000
1/1 [==============

...

Epoch 1047/10000
1/1 [==============================] - 0s 14ms/step - loss: 35014.9336 - val_loss: 2332.0044
Epoch 1048/10000
1/1 [==============================] - 0s 14ms/step - loss: 29755.5059 - val_loss: 1612.3646
Epoch 1049/10000
1/1 [==============================] - 0s 13ms/step - loss: 25854.6875 - val_loss: 1770.0912
Epoch 1050/10000
1/1 [==============================] - 0s 13ms/step - loss: 23519.7344 - val_loss: 2173.7205
Epoch 1051/10000
1/1 [==============================] - 0s 13ms/step - loss: 24458.7207 - val_loss: 2220.1260
Epoch 1052/10000
1/1 [==============================] - 0s 13ms/step - loss: 24281.5469 - val_loss: 2070.6204
Epoch 1053/10000
1/1 [==============================] - 0s 14ms/step - loss: 26848.7109 - val_loss: 2993.9761
Epoch 1054/10000
1/1 [==============================] - 0s 13ms/step - loss: 24070.2324 - val_loss: 2333.0796
Epoch 1055/10000
1/1 [==============================] - 0s 14ms/step - loss: 23209.8809 - val_loss: 2418.7537
Epoch 1056

1/1 [==============================] - 0s 13ms/step - loss: 36676.1289 - val_loss: 7653.7197
Epoch 1122/10000
1/1 [==============================] - 0s 13ms/step - loss: 31487.9121 - val_loss: 10063.3730
Epoch 1123/10000
1/1 [==============================] - 0s 13ms/step - loss: 39783.3398 - val_loss: 9831.1943
Epoch 1124/10000
1/1 [==============================] - 0s 13ms/step - loss: 30087.7207 - val_loss: 12083.3574
Epoch 1125/10000
1/1 [==============================] - 0s 13ms/step - loss: 29534.5938 - val_loss: 9864.7393
Epoch 1126/10000
1/1 [==============================] - 0s 14ms/step - loss: 24338.6719 - val_loss: 8911.6006
Epoch 1127/10000
1/1 [==============================] - 0s 13ms/step - loss: 26678.1035 - val_loss: 10298.5459
Epoch 1128/10000
1/1 [==============================] - 0s 13ms/step - loss: 25193.6953 - val_loss: 3363.7637
Epoch 1129/10000
1/1 [==============================] - 0s 14ms/step - loss: 27902.5938 - val_loss: 5641.1304
Epoch 1130/10000
1/1 [==

1/1 [==============================] - 0s 13ms/step - loss: 28835.9062 - val_loss: 11548.0684
Epoch 1196/10000
1/1 [==============================] - 0s 13ms/step - loss: 31383.7109 - val_loss: 8069.6436
Epoch 1197/10000
1/1 [==============================] - 0s 14ms/step - loss: 25577.6816 - val_loss: 8023.3760
Epoch 1198/10000
1/1 [==============================] - 0s 14ms/step - loss: 27525.3730 - val_loss: 8189.2334
Epoch 1199/10000
1/1 [==============================] - 0s 13ms/step - loss: 31012.0879 - val_loss: 6359.7710
Epoch 1200/10000
1/1 [==============================] - 0s 14ms/step - loss: 30646.7676 - val_loss: 5214.5029
Epoch 1201/10000
1/1 [==============================] - 0s 14ms/step - loss: 27690.0254 - val_loss: 2611.1819
Epoch 1202/10000
1/1 [==============================] - 0s 14ms/step - loss: 24938.5781 - val_loss: 3971.1052
Epoch 1203/10000
1/1 [==============================] - 0s 13ms/step - loss: 35647.8711 - val_loss: 4807.3120
Epoch 1204/10000
1/1 [====

1/1 [==============================] - 0s 13ms/step - loss: 33174.4648 - val_loss: 7981.1450
Epoch 1270/10000
1/1 [==============================] - 0s 14ms/step - loss: 28600.3887 - val_loss: 6840.9663
Epoch 1271/10000
1/1 [==============================] - 0s 14ms/step - loss: 32240.3750 - val_loss: 7859.5283
Epoch 1272/10000
1/1 [==============================] - 0s 14ms/step - loss: 25724.4863 - val_loss: 8555.7871
Epoch 1273/10000
1/1 [==============================] - 0s 14ms/step - loss: 28682.8027 - val_loss: 7003.0645
Epoch 1274/10000
1/1 [==============================] - 0s 13ms/step - loss: 26678.4512 - val_loss: 7377.7358
Epoch 1275/10000
1/1 [==============================] - 0s 13ms/step - loss: 29862.9395 - val_loss: 5781.0200
Epoch 1276/10000
1/1 [==============================] - 0s 13ms/step - loss: 29729.2812 - val_loss: 6202.2031
Epoch 1277/10000
1/1 [==============================] - 0s 13ms/step - loss: 32347.9316 - val_loss: 3598.2358
Epoch 1278/10000
1/1 [=====

1/1 [==============================] - 0s 13ms/step - loss: 30235.6816 - val_loss: 1465.1350
Epoch 1344/10000
1/1 [==============================] - 0s 13ms/step - loss: 25358.0840 - val_loss: 2767.3921
Epoch 1345/10000
1/1 [==============================] - 0s 14ms/step - loss: 26693.2441 - val_loss: 1698.3755
Epoch 1346/10000
1/1 [==============================] - 0s 13ms/step - loss: 30981.5410 - val_loss: 5806.4111
Epoch 1347/10000
1/1 [==============================] - 0s 14ms/step - loss: 28495.1152 - val_loss: 2827.3940
Epoch 1348/10000
1/1 [==============================] - 0s 14ms/step - loss: 28517.2324 - val_loss: 3258.5869
Epoch 1349/10000
1/1 [==============================] - 0s 13ms/step - loss: 27828.7949 - val_loss: 4300.9028
Epoch 1350/10000
1/1 [==============================] - 0s 14ms/step - loss: 24290.4160 - val_loss: 3555.8542
Epoch 1351/10000
1/1 [==============================] - 0s 13ms/step - loss: 25313.7246 - val_loss: 6478.2393
Epoch 1352/10000
1/1 [=====

In [31]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 35ms/step
[1788.146](test_target) - [[1452.1458]](prediction) = 336.00024804687496


In [32]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [33]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado,Mato Grosso - IDH,Mato Grosso - IDH Longevidade,Mato Grosso - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Mato Grosso - IDH Renda,Mato Grosso - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-0.958981,-1.053327,-0.731205,-0.792708,-1.266344,-1.367241,-1.240125,1.148205,-1.122043,-1.121418,...,1.412346,1.274172,-0.544470,1.358778,1.020728,1.401690,1.413631,1.261025,1.256285,0.043458
1,-0.420657,-0.290573,-0.682732,-0.617899,0.087939,0.370587,0.031362,0.140893,-0.184474,-0.185492,...,-0.643250,-0.105707,1.402573,-0.339841,0.337331,-0.538217,-0.741954,-0.076111,-0.065726,-1.245895
2,1.379638,1.343901,1.413937,1.410608,1.178406,0.996653,1.208763,-1.289098,1.306518,1.306910,...,-0.769096,-1.168466,-0.858104,-1.018937,-1.358059,-0.863473,-0.671678,-1.184915,-1.190558,1.202438


train_target:


,Mato Grosso - Consumo de Cimento (t)
0,998.465
1,1099.340
2,1204.074


test_input:


...

,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado,Mato Grosso - IDH,Mato Grosso - IDH Longevidade,Mato Grosso - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Mato Grosso - IDH Renda,Mato Grosso - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,0.765494,-0.551552,0.800035,0.794933,-1.569273,-2.551374,0.831359,1.108405,2.635823,-0.189959,...,-2.260447,-1.55722,-1.265069,-0.730185,0.611707,1.620843,2.316005,-1.810579,-2.274914,0.168633


test_target:


,Mato Grosso - Consumo de Cimento (t)
12,1788.146


1/1 [==============================] - 0s 36ms/step
Error: 336.00024804687496




[1183.2198486328125,
 1292.8214111328125,
 1401.4183349609375,
 1445.07861328125,
 1253.454345703125,
 1136.596923828125,
 1146.5283203125,
 1235.0997314453125,
 1421.920166015625,
 1452.145751953125]

In [34]:
display(targets)
display(predictions)

[1381.539,
 1618.657,
 1411.2344285714282,
 1241.1003428571426,
 1083.1282285714285,
 1220.058,
 1316.386,
 1446.064,
 1555.266,
 1788.146]

[1183.2198486328125,
 1292.8214111328125,
 1401.4183349609375,
 1445.07861328125,
 1253.454345703125,
 1136.596923828125,
 1146.5283203125,
 1235.0997314453125,
 1421.920166015625,
 1452.145751953125]

In [35]:
mae = mean_absolute_error(predictions, targets)
mae

184.19043278459822

In [36]:
porcentage = mae/np.mean(targets)
porcentage

0.1309884421831988